In [1]:
import numpy as np
import torch

# PyTorchテンソルの出力表示を見やすく設定
torch.set_printoptions(
    edgeitems=2,  # 各次元の先頭・末尾2個の要素だけ表示（中間は "..." で省略）
    threshold=50,  # 要素数がこれを超えると省略表示になる
    linewidth=75,  # 1行の最大表示幅。これを超えると自動改行される
)

In [ ]:
# CSVファイルからバイクシェアデータ（時間単位）を読み込み
bikes_numpy = np.loadtxt(
    "../../data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
    dtype=np.float32,  # 全データを float32 型として読み込む
    delimiter=",",  # カンマ区切りのCSVファイル
    skiprows=1,  # ヘッダー行をスキップ
    converters={
        1: lambda x: float(x[8:10])  # <1> 2列目（index=1）を "日付から日だけ抽出"
        # 例: "2011-01-01" → x[8:10] → "01" → float → 1.0
    },
)

# NumPy配列からPyTorchテンソルへ変換
bikes = torch.from_numpy(bikes_numpy)

# 結果を表示（テンソルの形状や内容の確認）
bikes

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

In [5]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [6]:
# bikes: shape = [17520, 17]（17520時間分、各時点17特徴量）

# 1日24時間ごとに区切って reshape（view）→ [日数, 24時間, 特徴量]
daily_bikes = bikes.view(-1, 24, bikes.shape[1])

# 結果の形状とストライドを確認
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [7]:
# daily_bikes: shape = (730, 17, 24)
# これは [日数, 特徴量数, 時間数] の構造で、各日について各特徴量の時間変化を表している

# transpose(1, 2) により、特徴量軸（1）と時間軸（2）を入れ替える
# 結果：shape = (730, 24, 17) となり、
# 各日について「時間ごとの特徴量ベクトル」が並ぶ構造に変わる
# これは例えば RNN で時間ごとの入力を処理したい場合などに便利な形
daily_bikes = daily_bikes.transpose(1, 2)

daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [8]:
# 1日目の24時間分のデータを抽出（行番号0〜23）
# bikes: shape = (17520, 17)
# 各行：1時間分のデータ（17特徴量）
# [:24] で最初の24時間（1日分）を抜き出す
first_day = bikes[:24].long()

# 天候IDは9列目に格納されており、0〜3の整数値
# この天候IDに対応するワンホットベクトルを作成するためのテンソルを定義
# shape: (24時間, 4種の天候クラス)
weather_onehot = torch.zeros(first_day.shape[0], 4)

# 天候情報（0〜3）を抽出
first_day[:, 9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2,
        2, 2])

In [9]:
# 天候IDは1〜4の整数で表されている（例: 1=晴れ, 2=曇り, 3=雨, 4=雪）
# それを0〜3に変換するために -1 をしてインデックス化（<1>）

# scatter_ を使って、ワンホットエンコーディングを実施：
# - dim=1: 各行の天候ID位置に 1.0 をセットする
# - index: shape = [24, 1] の天候インデックス（0〜3）に変換
# - value: その位置に代入する値（ここでは 1.0）
weather_onehot.scatter_(
    dim=1, index=first_day[:, 9].unsqueeze(1).long() - 1, value=1.0  # <1>
)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [14]:
# 最初の24時間分（1日）のデータ（shape: 24, 17）と、
# 対応する天候のワンホットベクトル（shape: 24, 4）を列方向に連結し、
# 拡張した特徴量ベクトル（shape: 24, 21）を得る
# そのうち最初の1時間（1行）のデータのみを抽出
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          6.0000,  0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,
          3.0000, 13.0000, 16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [15]:
# 日ごとの天候をワンホット形式で保持するためのテンソルを作成
# daily_bikes.shape = (日数, 時間数, 特徴量数)
# ここでは shape = (日数, 4, 特徴量数) のゼロテンソルを用意
# - 第1軸（dim=0）: 日数
# - 第2軸（dim=1）: 天候のクラス数（ワンホット用）
# - 第3軸（dim=2）: 特徴量方向へbroadcastするための軸
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])

# 形状確認：例 → torch.Size([730, 4, 17])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [16]:
# daily_bikes[:, 9, :] は、全日数（dim=0）に対する
# 「天候ID（9番目の特徴量）」×「24時間分（dim=2）」を抽出する
# 結果：shape = (730, 24) の2次元テンソル（整数の天候ID）

# `.unsqueeze(1)` によって shape = (730, 1, 24) に変形
# scatter_ の dim=1 で天候IDに対応する位置に 1.0 を代入する

daily_weather_onehot.scatter_(
    1,  # 書き込み対象：天候クラス軸
    daily_bikes[:, 9, :].long().unsqueeze(1) - 1,  # 天候ID（1〜4）→（0〜3）
    1.0,  # 代入する値
)

# 結果：各日の各時間に対応する天候IDを1-hot表現として記録
daily_weather_onehot.shape  # → torch.Size([730, 4, 17])

torch.Size([730, 4, 24])

In [20]:
# 特徴量テンソルに天候のワンホットを追加
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=2)

In [22]:
# 特徴量の9番目（天候ID：1〜4）を0〜1の範囲に正規化
# → 全日数・全時間帯で適用
daily_bikes[:, :, 9] = (daily_bikes[:, :, 9] - 1.0) / 3.0

In [23]:
# 特徴量10番目（気温）を抽出（日数×時間の2次元テンソル）
temp = daily_bikes[:, 10, :]

# 全体の最小・最大を取得（スカラー）
temp_min = torch.min(temp)
temp_max = torch.max(temp)

# Min-Max正規化: 全データの気温を0〜1にスケーリング
daily_bikes[:, 10, :] = (temp - temp_min) / (temp_max - temp_min)

In [24]:
# 特徴量10番目（気温）を抽出（shape: 730日 × 24時間）
temp = daily_bikes[:, 10, :]

# 気温データを標準化（平均0、標準偏差1にスケーリング）
daily_bikes[:, 10, :] = (temp - torch.mean(temp)) / torch.std(temp)